In [1]:
import pickle
import pandas as pd
import numpy as np

In [24]:
#import dataframe and drop nans
df_2021 = pd.read_csv('data/2021/2021_combined.csv', dtype={'Data':np.datetime64})

In [25]:
df_2021.describe().round()

,Unnamed: 0,Jaarmaand,Jaar,Maand,Weeknr,v15_1_Scheepstype_RWS,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw
count,385572.0,385572.0,385572.0,385572.0,385572.0,385572.0,385572.0,385572.0,278134.0,385572.0,274888.0
mean,208710.0,2107.0,2021.0,7.0,27.0,3.0,2446.0,5.0,1243360.0,15.0,6.0
std,120329.0,3.0,0.0,3.0,15.0,5.0,2521.0,3.0,1583801.0,51.0,3.0
min,7.0,2101.0,2021.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
25%,108799.0,2104.0,2021.0,4.0,14.0,1.0,1100.0,1.0,242000.0,0.0,3.0
50%,212580.0,2107.0,2021.0,7.0,26.0,1.0,1820.0,7.0,996000.0,0.0,6.0
75%,312478.0,2110.0,2021.0,10.0,40.0,2.0,3160.0,7.0,1700000.0,0.0,8.0
max,414145.0,2112.0,2021.0,12.0,53.0,36.0,100000.0,7.0,99999000.0,1700.0,9.0


In [26]:
#drop nans
df_2021_old = df_2021.copy()
df_2021 = df_2021.loc[df_2021.SK_CODE.isnull() != True]
x = len(df_2021_old)-len(df_2021)
print(x, "entries were because the SK_CODE entry was missing", x/len(df_2021_old), "%")

1995 entries were because the SK_CODE entry was missing 0.005174130901621487 %


In [27]:
print(len(df_2021), "trips are remaining")

383577 trips are remaining


In [28]:
df_2021.describe().round()

,Unnamed: 0,Jaarmaand,Jaar,Maand,Weeknr,v15_1_Scheepstype_RWS,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw
count,383577.0,383577.0,383577.0,383577.0,383577.0,383577.0,383577.0,383577.0,277585.0,383577.0,274351.0
mean,208786.0,2107.0,2021.0,7.0,27.0,3.0,2442.0,5.0,1242672.0,15.0,6.0
std,120331.0,3.0,0.0,3.0,15.0,5.0,2473.0,3.0,1571415.0,51.0,3.0
min,7.0,2101.0,2021.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
25%,108901.0,2104.0,2021.0,4.0,14.0,1.0,1100.0,1.0,243000.0,0.0,3.0
50%,212640.0,2107.0,2021.0,7.0,26.0,1.0,1820.0,7.0,996000.0,0.0,6.0
75%,312582.0,2110.0,2021.0,10.0,40.0,2.0,3160.0,7.0,1700000.0,0.0,8.0
max,414145.0,2112.0,2021.0,12.0,53.0,36.0,100000.0,7.0,95000000.0,1700.0,9.0


### Only five SK_CODE entries were missing, these entries were therefore dropped.

All other relevant data (date, origin and destination) were complete. No further action needed.

In [29]:
# group trips by origin, destination and category
df_t = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming', 'SK_CODE']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :4].reset_index(drop=True)

In [30]:
# give columns english names
df_t.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

In [31]:
# subset inland trips
df_t_old = df_t.copy()
df_t = df_t.loc[(df_t.origin.str.contains('NL')) & (df_t.destination.str.contains('NL'))]
df_t.reset_index(inplace=True, drop=True)

x = df_t_old.trips.sum()-df_t.trips.sum()
print(x, "non-inland trips were removed from the data set", x/len(df_2021_old), "%")

200959 non-inland trips were removed from the data set 0.5211970786260413 %


In [32]:
#create dict to fill with total number of trips on path for each ship type
df_t_dict = {}
# now fill this dict
# loop over data frame
for i in range(len(df_t)):
    #define temp variables for ease
    origin = df_t['origin'][i]
    destination = df_t['destination'][i]
    ship_type = df_t['type'][i]
    trip_count = df_t['trips'][i]
    #only make new column if trip and its inverse are not in dict yet. Otherwise append.
    if (origin, destination, ship_type) in df_t_dict.keys():
        df_t_dict[(origin, destination, ship_type)] += trip_count
    elif (destination, origin, ship_type) in df_t_dict.keys():
        df_t_dict[(destination, origin, ship_type)] += trip_count
    else:
        df_t_dict[(origin, destination, ship_type)] = trip_count
df_t_dict

{('NLAMS', 'NLRTM', 'M8'): 3510,
 ('NLAMS', 'NLRTM', 'M12'): 2076,
 ('NLRTM', 'NLMOE', 'M8'): 1584,
 ('NLRTM', 'NLRTM', 'M8'): 715,
 ('NLAMS', 'NLRTM', 'M9'): 1316,
 ('NLRTM', 'NLWLK', 'M3'): 973,
 ('NLOOS', 'NLTLB', 'M8'): 688,
 ('NLRTM', 'NLAMS', 'M6'): 881,
 ('NLAER', 'NLAMS', 'BI'): 824,
 ('NLDHR', 'NLDHR', 'M0'): 411,
 ('NLTLB', 'NLRTM', 'M8'): 623,
 ('NLRTM', 'NLOOS', 'M8'): 727,
 ('NLHAR', 'NLWTE', 'M12'): 690,
 ('NLHGL', 'NLRTM', 'M8'): 652,
 ('NLRTM', 'NLVLI', 'M8'): 658,
 ('NLRTM', 'NLRTM', 'M12'): 363,
 ('NLMOE', 'NLOOS', 'M8'): 615,
 ('NLVEG', 'NLRTM', 'M6'): 575,
 ('NLAMS', 'NLRTM', 'M11'): 666,
 ('NLMOE', 'NLAPN', 'M8'): 604,
 ('NLVEG', 'NLRTM', 'M7'): 581,
 ('NLAMS', 'NLMOE', 'M8'): 588,
 ('NLRTM', 'NLMEP', 'M8'): 568,
 ('NLSTD', 'NLSTD', 'M0'): 282,
 ('NLAMS', 'NLLMR', 'BIIL-1'): 553,
 ('NLNIJ', 'NLWAS', 'M8'): 508,
 ('NLWSP', 'NLAER', 'M6'): 510,
 ('NLDOR', 'NLRTM', 'M8'): 459,
 ('NLAMS', 'NLHAS', 'M8'): 436,
 ('NLRTM', 'NLDZL', 'M8'): 415,
 ('NLRTM', 'NLTIE', 'M8'): 4

## Compute vessel sum per route
Idea: make similar dict as above first, but then with total amount of trips per route. Later on: add column for each ship.
This will result in a column for each ship type and an entry with the amount of ships of that type that have travelled.

In [33]:
# now only sort by origin and destination
all_trips_ranked = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :3].reset_index(drop=True)

#again, rename columns
all_trips_ranked.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

#remove non nl
all_trips_ranked = all_trips_ranked.loc[(all_trips_ranked.origin.str.contains('NL')) & (all_trips_ranked.destination.str.contains('NL'))]

#reset index to be able to loop over all values later on
all_trips_ranked.reset_index(inplace=True, drop=True)

In [34]:
# make dict to store trips
trip_dict_main = {}

# now fill this dict, same procedure as above
for i in range(len(all_trips_ranked)):
    origin = all_trips_ranked['origin'][i]
    destination = all_trips_ranked['destination'][i]
    trip_count = all_trips_ranked['trips'][i]

    if (origin, destination) in trip_dict_main.keys():
        trip_dict_main[(origin, destination)] += trip_count
    elif (destination, origin) in trip_dict_main.keys():
        trip_dict_main[(destination, origin)] += trip_count
    else:
        trip_dict_main[(origin, destination)] = trip_count

# now sort values
trip_dict_main = dict(sorted(trip_dict_main.items(), key=lambda item: item[1], reverse=True))

In [35]:
trip_dict_main

{('NLRTM', 'NLAMS'): 9943,
 ('NLRTM', 'NLMOE'): 2686,
 ('NLRTM', 'NLVLI'): 2153,
 ('NLAMS', 'NLAER'): 2119,
 ('NLRTM', 'NLRTM'): 1794,
 ('NLVEG', 'NLRTM'): 1484,
 ('NLTLB', 'NLRTM'): 1318,
 ('NLLEY', 'NLAMS'): 1292,
 ('NLOSS', 'NLRTM'): 1145,
 ('NLZWO', 'NLAMS'): 1118,
 ('NLUTC', 'NLRTM'): 1042,
 ('NLRTM', 'NLKAM'): 1036,
 ('NLHGL', 'NLRTM'): 1035,
 ('NLLMR', 'NLAMS'): 1005,
 ('NLRTM', 'NLWLK'): 987,
 ('NLAMS', 'NLMOE'): 971,
 ('NLAER', 'NLRTM'): 945,
 ('NLRTM', 'NLAPN'): 945,
 ('NLTNZ', 'NLVLI'): 926,
 ('NLDOR', 'NLRTM'): 911,
 ('NLAMS', 'NLDEV'): 847,
 ('NLGOR', 'NLRTM'): 843,
 ('NLRTM', 'NLNIJ'): 817,
 ('NLBZM', 'NLRTM'): 814,
 ('NLWSP', 'NLAER'): 794,
 ('NLRTM', 'NLOOS'): 785,
 ('NLRTM', 'NLTNZ'): 779,
 ('NLVLI', 'NLAMS'): 769,
 ('NLRTM', 'NLMEP'): 768,
 ('NLOOS', 'NLTLB'): 737,
 ('NLNIJ', 'NLWAS'): 725,
 ('NLKAM', 'NLAMS'): 721,
 ('NLIJM', 'NLMOE'): 718,
 ('NLHAR', 'NLWTE'): 714,
 ('NLKGZ', 'NLAMS'): 696,
 ('NLBON', 'NLRTM'): 680,
 ('NLMOE', 'NLAPN'): 667,
 ('NLDOR', 'NLAMS'): 662

In [36]:
# make dataframe dict with a key for each column
trip_dict_df_main = {'origin': [], 'destination': [], 'trip_count': []}
for i in df_2021.SK_CODE.unique():
    trip_dict_df_main[i] = []

# now fill this dict
for key, value in trip_dict_main.items():
    # append values from dict a
    trip_dict_df_main['origin'].append(key[0])
    trip_dict_df_main['destination'].append(key[1])
    trip_dict_df_main['trip_count'].append(value)
    #now append values from other dict, for each ship type
    for ship_type in df_2021.SK_CODE.unique():
        #if ships have travelled from A to B, append this value
        if (key[0], key[1], ship_type) in df_t_dict.keys():
            trip_dict_df_main[ship_type].append(df_t_dict[(key[0], key[1], ship_type)])
        elif (key[1], key[0], ship_type) in df_t_dict.keys():
            trip_dict_df_main[ship_type].append(df_t_dict[(key[1], key[0], ship_type)])
        else:
            # no? Then no ships of this type have travelled this path, append 0
            trip_dict_df_main[ship_type].append(0)

KeyboardInterrupt: 

In [ ]:
df_main = pd.DataFrame.from_dict(trip_dict_df_main)

In [ ]:
df_main

In [ ]:
print(len(df_main), "O-D pairs were initially found for a total of", sum(df_main.trip_count), "trips in 2021")

In [ ]:
#remove entries with same origin and destination
df_main_old = df_main.copy()
df_main = df_main.loc[df_main.origin!=df_main.destination]
print(len(df_main_old)-len(df_main), "O-D pairs were removed because they were loops with same origin and dest")

In [ ]:
print(sum(df_main.trip_count), "trips are still left in the data set")

In [ ]:
print(182618-176478, "were thus self loops")

In [ ]:
df_main

In [ ]:
#save dataframe
# pickle.dump( df_main, open( "data/df_trips_per_path.p", "wb" ) )
df_main = pickle.load( open( "data/df_trips_per_path.p", "rb" ) )

In [ ]:
df_main

In [ ]:
print(len(df_main), "OD-pairs are left for a total of", sum(df_main.trip_count), "trips in 2021")

## Compute vessel sum per route and per hour (for ABM)
Only for top 100 origin destination pairs

In [ ]:
#generate hour
df_2021['hour'] = df_2021.Date.apply(lambda x: int(x.split()[1][:2]))

In [ ]:
# now only sort by origin and destination
all_trips_ranked = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming', 'hour', 'SK_CODE']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :5].reset_index(drop=True)

#again, rename columns
all_trips_ranked.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

#remove non nl
all_trips_ranked = all_trips_ranked.loc[
    (all_trips_ranked.origin.str.contains('NL')) & (all_trips_ranked.destination.str.contains('NL'))]

#reset index to be able to loop over all values later on
all_trips_ranked.reset_index(inplace=True, drop=True)

In [ ]:
all_trips_ranked

In [ ]:
# make dict to store trips
trip_dict_main = {}

# now fill this dict, same procedure as above
for i in range(len(all_trips_ranked)):
    origin = all_trips_ranked['origin'][i]
    destination = all_trips_ranked['destination'][i]
    trip_count = all_trips_ranked['trips'][i]
    hour = all_trips_ranked['hour'][i]
    ship_type = all_trips_ranked['type'][i]

    if (origin, destination, ship_type, hour) in trip_dict_main.keys():
        trip_dict_main[(origin, destination, ship_type, hour)] += trip_count
    elif (destination, origin, ship_type, hour) in trip_dict_main.keys():
        trip_dict_main[(destination, origin, ship_type, hour)] += trip_count
    else:
        trip_dict_main[(origin, destination, ship_type, hour)] = trip_count
# now sort values
trip_dict_main = dict(sorted(trip_dict_main.items(), key=lambda item: item[1], reverse=True))



In [ ]:
trip_dict_main

In [ ]:
# now only keep entries of trip_dict_main that are part of top 100 OD pairs
top100 = df_main.head(100)
top100.reset_index(drop=True, inplace=True)
ordes = []
for i in range(len(top100)):
    ordes.append((top100['origin'][i], top100['destination'][i]))

trip_dict_subset = {i:trip_dict_main[i] for i in trip_dict_main.keys() if (i[0],i[1]) in ordes}

In [ ]:
print("remaining:", top100.trip_count.sum(), "dropped", df_main.trip_count.sum()-top100.trip_count.sum())

In [ ]:
len(trip_dict_subset.keys())

In [ ]:
df_main.loc[(df_main.origin == 'NLRTM')&(df_main.destination == 'NLAMS')].trip_count.item()

In [ ]:
# make dataframe dict wih a key for each column
trip_dict_df_main = {'origin': [], 'destination': [], 'hour':[]}
for i in df_2021.SK_CODE.unique():
    trip_dict_df_main[i] = []

# now fill this dict
for key in ordes:
    # append values from dict a
    for hour in list(np.arange(0,24,1)):
        trip_dict_df_main['origin'].append(key[0])
        trip_dict_df_main['destination'].append(key[1])
        trip_dict_df_main['hour'].append(hour)
        #now append values from other dict, for each ship types
        for ship_type in df_2021.SK_CODE.unique():
            #if ships have travelled from A to B, append this value
            if (key[0], key[1], ship_type, hour) in trip_dict_main.keys():
                trip_dict_df_main[ship_type].append(trip_dict_main[(key[0], key[1], ship_type, hour)])
            elif (key[1], key[0], ship_type, hour) in trip_dict_main.keys():
                trip_dict_df_main[ship_type].append(trip_dict_main[(key[1], key[0], ship_type, hour)])
            else:
                # no? Then no ships of this type have travelled this path, append 0
                trip_dict_df_main[ship_type].append(0)
df_main_hourly = pd.DataFrame.from_dict(trip_dict_df_main)
df_main_hourly

In [ ]:
df_main_hourly.insert(2, 'trip_count', df_main_hourly.iloc[:, 3:].sum(axis=1))

In [ ]:
df_main_hourly

In [ ]:
pickle.dump( df_main_hourly, open( "ABM/own_work/data/df_trips_per_path_hourly.p", "wb" ) )

## Now translate into fuel consumption value for each path
Idea: take 1 vessel as a basis, and assume that fuel consumption is linearly correlated with average engine power, to be able to use index
Capacity is likely x vessels per day of a certain type for a charging unit, depending on the charging time.
Hence, computing the average amount of that type of vehicles that comes by everyday makes sense.

### First determine factors

In [2]:
ship_data = pd.read_csv('data/Vessel-database-2.csv')
ship_data.avv_class = ship_data.avv_class.str.replace("_","-")
ship_data.avv_class = ship_data.avv_class.str.replace("BII-2l","BII-2L")
ship_data.fillna(0, inplace=True)
# only keep relevant columns
ship_data = ship_data.loc[:, ["P_installed", "speed_loaded", "speed_unloaded", "avv_class", "cemt_class"]]
ship_data.rename(columns={'avv_class':'RWS-class'}, inplace=True)

In [3]:
ship_data['Factor'] = ship_data.P_installed.apply(lambda n: (n/ship_data.P_installed[0]))

### Do we have a factor for each column?

In [4]:
# import final flow df
top100 = pickle.load(open("data/revised_cleaning_results/ivs_exploded_100.p", 'rb'))

In [5]:
# Check for which columns we do not have a factor
set(top100.columns[4:-1]) - set(ship_data['RWS-class'])

{'BIIL-1', 'BIIa-1', 'M0'}

In [6]:
# no columns in df without factor!
set(ship_data['RWS-class']) - set(top100.columns[4:-1])

set()

No, is this a problem?

In [7]:
# load final data set
print(top100.trip_count.sum())
top100.drop(columns=['BIIL-1', 'BIIa-1']).trip_count.sum()
print(top100.trip_count.sum())

73842
73842


No, so we can just drop these columns and continue!

In [8]:
top100 = top100.drop(columns=['BIIL-1', 'BIIa-1'])

In [9]:
ship_data = ship_data.loc[~ship_data['RWS-class'].isin(['BIIL-1', 'BIIa-1'])]

In [10]:
ship_data_factors = dict(zip(ship_data['RWS-class'],ship_data['P_installed']))

In [11]:
ship_data_factors

{'M1': 175.0,
 'M2': 250.0,
 'M3': 435.0,
 'M4': 435.0,
 'M5': 435.0,
 'M6': 690.0,
 'M7': 690.0,
 'M8': 1356.0,
 'M9': 1744.0,
 'M10': 1649.0,
 'M11': 2021.0,
 'M12': 2233.0,
 'B01': 100.0,
 'B02': 120.0,
 'B03': 150.0,
 'B04': 380.0,
 'BI': 738.0,
 'BII-1': 1238.0,
 'BII-2L': 1238.0,
 'BII-2b': 1238.0,
 'BII-4': 1238.0,
 'BII-6l': 1238.0,
 'BII-6b': 1238.0,
 'C1l': 175.0,
 'C1b': 175.0,
 'C2l': 690.0,
 'C3l': 1425.0,
 'C2b': 690.0,
 'C3b': 1425.0,
 'C4': 1425.0}

In [12]:
pickle.dump(ship_data_factors, open('data/flow_comp_factors_unscaled.p', 'wb'))
pickle.dump(ship_data, open('data/full_ship_data.p', 'wb'))
ship_data.to_excel("data/ship_types.xlsx")

### Now use factors for flow computation per route

In [13]:
from flow_computation import flow_computation

In [14]:
flows = flow_computation(top100)

In [15]:
flows

{('NLABL', 'NLRTM', 0): 11.955882583170258,
 ('NLAER', 'NLLEY', 0): 4.3540508806262235,
 ('NLAER', 'NLNIE', 0): 7.4868571428571435,
 ('NLAER', 'NLRTM', 0): 16.19282974559687,
 ('NLAER', 'NLSWI', 0): 2.7790998043052846,
 ('NLALK', 'NLIJM', 0): 3.626019569471625,
 ('NLALM', 'NLAMS', 0): 3.8248297455968685,
 ('NLAMS', 'NLAER', 0): 21.148007827788646,
 ('NLAMS', 'NLDEV', 0): 6.470027397260275,
 ('NLAMS', 'NLHEJ', 0): 2.964430528375734,
 ('NLAMS', 'NLMOE', 0): 0.02332681017612525,
 ('NLAMS', 'NLMOE', 1): 0.4418003913894326,
 ('NLAMS', 'NLMOE', 2): 1.8949197651663408,
 ('NLAMS', 'NLMOE', 3): 13.425941291585128,
 ('NLAMS', 'NLMOE', 4): 1.5602348336594911,
 ('NLAMS', 'NLOSS', 0): 5.208250489236792,
 ('NLAMS', 'NLTIE', 0): 4.484962818003914,
 ('NLAMS', 'NLUTC', 0): 0.0,
 ('NLAMS', 'NLUTC', 1): 6.515694716242662,
 ('NLAMS', 'NLWGW', 0): 4.3261056751467715,
 ('NLAMS', 'NLWMO', 0): 2.2882035225048925,
 ('NLAMS', 'NLZAA', 0): 2.9664187866927594,
 ('NLAMS', 'NLZWI', 0): 0.08610567514677106,
 ('NLAMS

In [16]:
#sort flows from large to small
flows = dict(sorted(flows.items(), key=lambda item: item[1], reverse=True))

In [17]:
flows

{('NLRTM', 'NLAMS', 1): 123.24020352250488,
 ('NLRTM', 'NLAMS', 2): 111.82943248532288,
 ('NLRTM', 'NLVLI', 0): 47.688516634050906,
 ('NLRTM', 'NLMOE', 0): 41.60410176125244,
 ('NLHAR', 'NLWTE', 0): 24.280328767123287,
 ('NLAMS', 'NLAER', 0): 21.148007827788646,
 ('NLRTM', 'NLKAM', 2): 20.602724070450098,
 ('NLVLI', 'NLAMS', 1): 20.06252837573386,
 ('NLUTC', 'NLRTM', 0): 19.03783953033268,
 ('NLGOR', 'NLRTM', 0): 18.137816046966734,
 ('NLHGL', 'NLRTM', 0): 17.359561643835615,
 ('NLTLB', 'NLRTM', 1): 17.295624266144817,
 ('NLNIJ', 'NLWAS', 0): 16.438090019569476,
 ('NLAER', 'NLRTM', 0): 16.19282974559687,
 ('NLRTM', 'NLNIJ', 0): 16.16954990215264,
 ('NLRTM', 'NLOOS', 1): 16.140461839530335,
 ('NLRTM', 'NLTNZ', 0): 15.903436399217226,
 ('NLTNZ', 'NLVLI', 0): 15.589166340508807,
 ('NLNIJ', 'NLABL', 0): 15.056219178082193,
 ('NLOOS', 'NLTLB', 0): 14.915679060665363,
 ('NLVEG', 'NLRTM', 0): 14.899334637964772,
 ('NLBON', 'NLRTM', 1): 14.174794520547946,
 ('NLZWO', 'NLAMS', 0): 14.0847749510

## Get and clean harbour data NL, Ger and Bel

In [ ]:
# now get coordinates harbours, in three files, check encoding frist below
pd.read_csv('data/harbour_codes_coords/2021-2 UNLOCODE CodeListPart1.csv', encoding="ISO-8859-1", header=None, sep=",",
            index_col=None)

In [ ]:
# now identify all files to combine
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('data/harbour_codes_coords'):
    f.extend(filenames)
    break

In [ ]:

#empty list to store data
li = []

#read everything in as object for now
for filename in f:
    df = pd.read_csv("data/harbour_codes_coords/" + str(filename), sep=',', index_col=None, header=None,
                     encoding="ISO-8859-1")
    print(filename, 'handled')
    li.append(df)

In [ ]:
#create df
df_harbour_codes = pd.concat(li, axis=0, ignore_index=True)
df_benelux=df_harbour_codes.copy()
#subset Dutch harbours
# df_benelux = df_harbour_codes.loc[(df_harbour_codes[1] == 'NL')]

# uncomment below to loc benelux, for now only use NL because of scope
# df_benelux = df_harbour_codes.loc[
#     (df_harbour_codes[1] == 'NL') | (df_harbour_codes[1] == 'DE') | (df_harbour_codes[1] == 'BE') | (
#                 df_harbour_codes[1] == 'LU')]

In [ ]:
#fillna
df_benelux.fillna(np.nan, inplace=True)

In [ ]:
# give appropriate column names
df_benelux.rename(columns={0: 'mutation', 1: 'country', 2: 'city_abbr', 3: 'city_full', 4: 'city_full_2', 5: 'subdivision', 6: 'function', 7:'status', 8:'date', 9:'iata_code', 10: 'coords', 11: 'comments'}, inplace=True)

In [ ]:
df_benelux

In [ ]:
# subset relevant columns
# df_benelux = df_benelux.loc[:,['country','city_abbr','function', 'coords']]

In [ ]:
#reset index
df_benelux.reset_index(inplace=True,drop=True)

In [ ]:
# drop rows with nan values for necessary columns
# df_benelux.dropna(subset=['country', 'city_abbr', 'coords'], inplace = True)

In [ ]:
# # convert columns to be normal strings
# for i in df_benelux.columns:
#     df_benelux[i] = df_benelux[i].astype('|S80')
#     df_benelux[i] = df_benelux[i].apply(lambda x: x.decode('utf-8'))

In [ ]:
df_benelux.replace('nan', np.nan)

In [ ]:
df_benelux['harbour_code'] = df_benelux.country+df_benelux.city_abbr

In [ ]:
df_benelux = df_benelux.loc[:,['country','city_abbr','city_full','function','coords','harbour_code']]

In [ ]:
# save cleaned data
df_benelux.to_csv('data/cleaned_harbours.csv')